### first step:     
#### find the redundant layers[very little information is converted]

In [1]:
# tensorflow
import tensorflow as tf
import torch
import os
import pandas as pd
from tensorflow.keras.datasets import cifar10
os.environ["CUDA_VISIBLE_DEVICES"] = "3" 
(x_train, y_train), (x_test, y_test) = cifar10.load_data() 
x_test = x_test/255.0
x_train = x_train/255.0
model = tf.keras.models.load_model('../checkpoint/')
optimizer = tf.keras.optimizers.SGD(0.001, momentum=0.9)
model.compile(
    optimizer,
    tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['acc'])
data = pd.read_pickle('../checkpoint/0.99_modules.pkl')
print('redundant blocks[when threshold is 0.99]: ',end=' ')
for d in data[2]:
    if d[0]!=d[1]:
        print(d,end=' ')

2022-10-14 00:39:15.294925: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-14 00:39:20.991653: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17387 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b5:00.0, compute capability: 8.6


redundant blocks[when threshold is 0.99]:  [5, 9] [22, 35] [38, 49] [53, 54] 

#### second step:

##### we can already get the redundant part of a trained neural network, and then we can compress it according to the loss of accuracy that we can accept.
##### Note that: our compression here is used before any other compression tool, since our compression granularity is 'network layer or block'

In [2]:
# Suppose the redundant blocks to be processed are [46,47,48,49]
os.system('python find_redundant.py --experiment_dir ../checkpoint/ --layer_indexes 46 47 48 49')

2022-10-14 00:39:47.912214: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-14 00:39:53.642510: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17387 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b1:00.0, compute capability: 8.6


[46, 47, 48, 49]


2022-10-14 00:40:17.399706: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8204
2022-10-14 00:40:25.501245: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


After deleting 1 blocks, the guaranteed accuracy is 0.9372 
After deleting 2 blocks, the guaranteed accuracy is 0.9344 
After deleting 3 blocks, the guaranteed accuracy is 0.9259 
After deleting 4 blocks, the guaranteed accuracy is 0.9135 


0

#### third step:
##### Used other compression tools you need on the pre-compressed network
